In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Exploratory Data Analysis of Spotify Songs

* Description - I have been using spotify since it arrived in India and this is my one of the most trusted music platform on which i listen music everyday approx 4-5 hrs. Recently i discovered that Spotify uses Machine Learning to suggest songs to its users according to their mood or past listenings. So i thought of doing this exploratory data analysis. since this data is huge i am using only tracks.csv data for this. because i don't wanna dig that deep into it. 
* The Main Objective of doing this analysis is i want to find the trends based on people the music is divided like according to popularity or year it was released.

*Lets Begin

<img src = "https://sm.mashable.com/mashable_in/news/s/spotify-re/spotify-releases-top-streamed-songs-artists-and-podcasts-for_xvnc.png"/>

In [ ]:
#import the required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
#use plotly locally
import plotly.graph_objs as go

## Import Dataset

In [ ]:
data_df = pd.read_csv('../input/spotify-dataset-19212020-160k-tracks/tracks.csv')

In [ ]:
#we are sorting the data according to their popularity 
sorted_df = data_df.sort_values('popularity', ascending = False).head(100)
sorted_two_df = data_df.sort_values('popularity', ascending = True)
date = list(data_df.release_date)
for i in range(len(date)):
    date[i] = int(date[i][:4])

In [ ]:
data_df['Year'] = date

In [ ]:
df_pre90 = data_df[data_df.Year <= 1990]
df_post_90 = data_df[data_df.Year > 1990]

In [ ]:
#lets see sorted head
sorted_df.head(10)

# Cleaning and Visualisation of Data

* Top 100 most popular songs vs. all of the rest - Features

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data_df.info()

In [ ]:
missing_percentage = pd.DataFrame(data_df.isnull().sum()*100/data_df.count())
missing_percentage.columns = ['Missing_Percentage']
missing_percentage.T

In [ ]:
#Checking null Value in the Dataset
data_df.isna().sum()

In [ ]:
sorted_df_1 = sorted_df[['explicit','danceability','energy','mode','speechiness','acousticness','instrumentalness','liveness','valence']]
sorted_two_df = sorted_two_df[['explicit','danceability','energy','mode','speechiness','acousticness','instrumentalness','liveness','valence']]

In [ ]:
labels = list(sorted_df_1)[:]

stats = sorted_df_1.mean().tolist()

stats2 = sorted_two_df.mean().tolist()

angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False)
fig = plt.figure(figsize = (15,15))

ax = fig.add_subplot(221, polar=True)
ax.plot(angles, stats, 'o-', linewidth=3, label = "Popular", color = 'green')
ax.fill(angles, stats, alpha=0.25, facecolor='green')
ax.set_thetagrids(angles * 180/np.pi, labels, fontsize = 12)

ax.set_rlabel_position(250)
plt.yticks([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7], ["0.1", "0.2", "0.3", "0.4", "0.5","0.6", "0.7" ], size=12)
plt.ylim(0,0.7)

ax.plot(angles, stats2, 'o-', linewidth=2, label = "All The Dataset", color= 'red')
ax.fill(angles, stats2, alpha=0.25, facecolor='red')
ax.set_title('Mean Values')
ax.grid(True)

plt.legend(loc='best', bbox_to_anchor=(0.1,0.1))

In [ ]:
import networkx as nx
from scipy import spatial

In [ ]:
list_songs = list(sorted_df.name)

In [ ]:
spatial.distance.cosine(sorted_df_1.iloc[1], sorted_df_1.iloc[0])

In [ ]:
dict_edges = {}
for i in range(len(list_songs)):
    list_val = []
    for j in range(i):
        a_row = list(sorted_df_1.iloc[j])
        list_val.append([j, 1 - spatial.distance.cosine(sorted_df_1.iloc[i], a_row),list(sorted_df.iloc[j])[1] ])
    for j in range(i+1,len(list_songs)):    
        a_row = list(sorted_df_1.iloc[j])
        list_val.append([j, 1 - spatial.distance.cosine(sorted_df_1.iloc[i], a_row), list(sorted_df.iloc[j])[1]])
    dict_edges[i] = list_val

In [ ]:
dict_edges[0][1]

In [ ]:
new_dict = {}
for i in range(100):
    list_to_put = []
    for j in range(len(dict_edges[i])):
        if dict_edges[i][j][1] >= 0.95:
            list_to_put.append(dict_edges[i][j])      
    new_dict[i] = list_to_put

In [ ]:
new_dict[6]

## Most Popular Artists

In [ ]:
from collections import Counter
import re

In [ ]:
sorted_df.artists.values[0] = sorted_df.artists.values[0].replace('[', '')
sorted_df.artists.values[0] = sorted_df.artists.values[0].replace(']', '')
sorted_df.artists.values[0]

r1 = re.compile(r"\w{1,20}.\w{1,20}", re.MULTILINE | re.DOTALL)
r = r1.findall(sorted_df.artists.values[0])

In [ ]:
list_top_100 = list(sorted_df.artists.values)


In [ ]:

all_artists = []
for i in range(len(list_top_100)):
    r1 = re.compile(r"\w{1,20}.\w{1,20}", re.MULTILINE | re.DOTALL)
    r = r1.findall(list_top_100[i])
    if len(r) > 0:
        all_artists.append(r[0])

In [ ]:
c = Counter(all_artists)
list(c)[:10]

In [ ]:
fig = plt.figure(figsize = (10,8))
sns.barplot(x = 'Name', y = 'Count', data = df_artists)
plt.xticks(rotation=90)

In [ ]:
plt.figure(figsize=(15,35))

songs_loudness_graph = sns.barplot(x=spotify_df["Loudness (dB)"],
                                   y=spotify_df["Year"],
                                   orient="h");

# Questions and Ask Answers

## Q1: Who are the Top Artists on Spotify 

In [ ]:
df_artists = pd.DataFrame(data = {'Name':list(c)[:20], 'Count':list(c.values())[:20]})

In [ ]:
fig = plt.figure(figsize = (10,8))
sns.barplot(x = 'Name', y = 'Count', data = df_artists)
plt.xticks(rotation=90)

* As we can see that Justin Beiber and Bad Bunny are top in the list as Top artists

## Q2: Songs Which have popularity less than 50%

In [ ]:
popularity_50=data_df[data_df.popularity>=50]
popularity_50

* These Songs have Popularity less than 50 %

## Q3: Songs Which have popularity more than 95%

In [ ]:
#lets Find Out Top Songs of Spotify 2020-2021 which has popularity greater than 95
popularity_95=data_df[data_df.popularity>95]
popularity_95

* Here are the top Songs of 2020-2021

## Q4: Songs Released in an Specific year( Show It on Graph)

In [ ]:
plt.figure(figsize=(15,35))

songs_year_graph = sns.countplot(y=data_df["Year"])

plt.xlabel("Number of Songs")
plt.title("Songs Released on Particular Year");

* As of Now 2020 was the year in which most songs were released on Spotify.

## - Q5: Most popular song of all time 

In [ ]:
plt.figure(figsize=(20,10))

sns.distplot(x=data_df["duration_ms"],
            kde=True);

plt.xlabel("Length (s)")
plt.title("Length (Duration) used in the Songs");

Reference:
    
    * Dataset: "https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks?select=tracks.csv"

<img src = "https://media.giphy.com/media/26gsjCZpPolPr3sBy/giphy.gif"/>